<a href="https://colab.research.google.com/github/keshav123333/new_york_taxi_fare_pred_ml_model/blob/main/machinelearningproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install opendatasets pandas numpy scikit-learn xgboost --quiet

In [ ]:
import opendatasets as od
dataset_url="https://www.kaggle.com/c/new-york-city-taxi-fare-prediction";
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: keshavrai2004
Your Kaggle Key: ··········


100%|██████████| 1.56G/1.56G [00:14<00:00, 114MB/s]



Extracting archive ./new-york-city-taxi-fare-prediction/new-york-city-taxi-fare-prediction.zip to ./new-york-city-taxi-fare-prediction


In [ ]:
data_dir="new-york-city-taxi-fare-prediction"

In [ ]:
!ls -lh {data_dir}

total 5.4G
-rw-r--r-- 1 root root  486 Mar  7 15:02 GCP-Coupons-Instructions.rtf
-rw-r--r-- 1 root root 336K Mar  7 15:02 sample_submission.csv
-rw-r--r-- 1 root root 960K Mar  7 15:02 test.csv
-rw-r--r-- 1 root root 5.4G Mar  7 15:02 train.csv


In [ ]:
!wc -l {data_dir}/train.csv

55423856 new-york-city-taxi-fare-prediction/train.csv


In [ ]:
!head {data_dir}/train.csv

key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.84161,40.712278,1
2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.76127,-73.991242,40.750562,2
2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.98713,40.733143,-73.991567,40.758092,1
2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.73163,-73.972892,40.758233,1
2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1
2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.9513,40.774138,-73.990095,40.751048,1
2012-12-03 13:10:00.000000125,9,2012-12-03 13:10:00 UTC,-74.006462,40.726713,-73.99

In [ ]:
import pandas as pd
import random
dtypes={
    "fare_amount":"float32",

    "pickup_longitude":"float32",
    "pickup_latitude":"float32",
    "dropoff_longitude":"float32",
    "dropoff_latitude":"float32",
    "passenger_count":"uint8"
}
def skip_row(row_index):
    if row_index==0:
        return False
    return random.random()>0.01
random.seed(42)
selected_col="key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count".split(",")
df=pd.read_csv(data_dir+"/train.csv",usecols=selected_col,parse_dates=["pickup_datetime"],skiprows=skip_row)

In [ ]:
test_df=pd.read_csv(data_dir+"/test.csv",parse_dates=["pickup_datetime"],dtype=dtypes)
test_df

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51+00:00,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51+00:00,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15+00:00,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19+00:00,-73.985573,40.735432,-73.939178,40.801731,6


In [ ]:
from sklearn.model_selection import train_test_split
train_df,val_df=train_test_split(df,test_size=0.2)
train_df.dropna()
val_df.dropna()


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
159723,2010-02-08 14:35:00.00000062,4.9,2010-02-08 14:35:00+00:00,-73.999677,40.734092,-73.999680,40.743688,1
101660,2014-11-30 15:02:52.0000003,6.5,2014-11-30 15:02:52+00:00,-73.988589,40.736370,-73.972829,40.736761,1
68459,2015-06-15 21:47:08.0000009,9.5,2015-06-15 21:47:08+00:00,-73.997780,40.721603,-73.980049,40.726723,1
242845,2013-04-14 18:53:00.00000065,11.5,2013-04-14 18:53:00+00:00,-73.977517,40.758130,-74.002990,40.733570,6
211219,2013-07-08 16:41:00.00000027,23.5,2013-07-08 16:41:00+00:00,-74.010105,40.720405,-73.959708,40.766922,2
...,...,...,...,...,...,...,...,...
213376,2014-10-09 09:24:29.0000001,4.5,2014-10-09 09:24:29+00:00,-73.985619,40.746945,-73.990062,40.737956,2
287884,2012-08-23 01:02:00.00000026,10.9,2012-08-23 01:02:00+00:00,-73.987635,40.719813,-73.954970,40.765605,4
472921,2013-11-23 22:46:00.00000077,5.0,2013-11-23 22:46:00+00:00,-73.985202,40.727802,-73.990837,40.724297,1
493378,2014-04-02 23:31:00.000000194,9.0,2014-04-02 23:31:00+00:00,-73.995498,40.721372,-73.983880,40.694762,1


In [ ]:
train_df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [ ]:
input_cols= [ 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count']
target_col="fare_amount"

train_inputs=train_df[input_cols]
train_targets=train_df[target_col]
val_inputs=val_df[input_cols]
val_targets=val_df[target_col]

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
def rmse(target,pred):
  return np.sqrt(mean_squared_error(target,pred ))

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model=LinearRegression()
linear_model.fit(train_inputs,train_targets)
linear_pred=linear_model.predict(train_inputs)

rmse(train_targets,linear_pred)

9.830233370071477

In [ ]:


print(linear_model.get_params())

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}


In [ ]:
def predict_and_submit(model,test_inputs,fname):
  test_preds=model.predict(test_inputs)
  sub_df=pd.read_csv(data_dir+"/sample_submission.csv")
  sub_df["fare_amount"]=test_preds
  sub_df.to_csv(fname,index=None)
  return sub_df

In [ ]:
def add_dateparts(df,col):
  df[col+"_year"]=df[col].dt.year
  df[col+"_month"]=df[col].dt.month
  df[col+"_day"]=df[col].dt.day
  df[col+"_weekday"]=df[col].dt.weekday
  df[col+"_hour"]=df[col].dt.hour

In [ ]:
add_dateparts(train_df,"pickup_datetime")
add_dateparts(val_df,"pickup_datetime")
add_dateparts(test_df,"pickup_datetime")


In [ ]:
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6378.137 * c
    return km

In [ ]:
def distance_from(df):
  df["trip_distance"]=haversine_np(df["pickup_longitude"],df["pickup_latitude"],df["dropoff_longitude"],df["dropoff_latitude"])



In [ ]:
distance_from(train_df)
distance_from(val_df)
distance_from(test_df)


In [ ]:
jik_lonlat = -73.7781, 40.6413
lga_lonlat = -73.8740, 40.7769
ewr_lonlat = -74.1745, 40.6895
met_lonlat = -73.9632, 40.7794
wte_lonlat = -74.0099, 40.7126

In [ ]:
def add_landmark_dropoff_distance(df,landmark_name,landmark_lonlat):
  lon,lat=landmark_lonlat
  df[landmark_name+"_drop_distance"]=haversine_np(lon,lat,df["dropoff_longitude"],df["dropoff_latitude"])


In [ ]:
def add_landmarks(df):
    landmarks = {
        "jfk": jik_lonlat,
        "lga": lga_lonlat,
        "ewr": ewr_lonlat,
        "met": met_lonlat,
        "wtc": wte_lonlat
    }

    for name, lonlat in landmarks.items():
        add_landmark_dropoff_distance(df, name, lonlat)

In [ ]:
add_landmarks(train_df)
add_landmarks(val_df)
add_landmarks(test_df)

In [ ]:
def remove_outliers(df):
    return df[
        (df['fare_amount'] > 1) & (df['fare_amount'] < 500) &
        (df['pickup_longitude'] >= -75) & (df['pickup_longitude'] <= -72) &
        (df['dropoff_longitude'] >= -75) & (df['dropoff_longitude'] <= -72) &
        (df['pickup_latitude'] >= 40) & (df['pickup_latitude'] <= 42) &
        (df['dropoff_latitude'] >= 40) & (df['dropoff_latitude'] <= 42) &
        (df['passenger_count'] >= 1) & (df['passenger_count'] <= 6)
    ]


In [ ]:
 #crete a list
lis=(2,3,4)
print(lis)

print


(2, 3, 4)
